In [1]:
#  Hidden Setup
import pandas as pd
import openai
from credentials import openai_api_key

# Read the first CSV file
file_path = '/Users/mkarreth/Downloads/comp.csv'
df1 = pd.read_csv(file_path)

# Read the second CSV file
df2 = pd.read_csv('/Users/mkarreth/Downloads/prod.csv')

# Set up OpenAI API credentials
openai.api_key = openai_api_key

# A simple LLM-Lookup
It's just the good old Excel VLOOKUP, but powered by LLMs. Specifically ever wondered about that approximate match functionality in a VLOOKUP? Well here we go for real.

## (1) Take two CSV files and read them into dataframes

In [2]:
companies = df1
website_products = df2
display(companies.describe())
display(website_products.describe())

,Company
count,10
unique,8
top,Apple
freq,2


,Website,Primary product
count,8,8
unique,8,8
top,apple.com,iPhone
freq,1,1


In [3]:
display(companies)
display(website_products)

,Company
0,Apple
1,LVMH
2,Tesla
3,Meta
4,Alphabet
5,Microsoft
6,Apple
7,Tesla
8,Ford
9,ENI Energy


,Website,Primary product
0,apple.com,iPhone
1,www.tesla.com,Cars
2,lvmh.com,Handbags
3,https://wework.com,Coworking
4,www.united.com,Flights
5,uniqlo.com,Fashion
6,www.eni.it,Energy
7,www.meta.com,Social media


## (2) Now loop through the first CSV and match to second CSV
The match, similar to a VLOOKUP, happens through a specified matching columns. In this case it is "Company" and "Website."

In [4]:
# Loop through each row in the first CSV and insert the values of the rows for columns x, y, and z into the prompt
for index, row in companies.iterrows():
    company_in_row = row['Company']
    website_products_values = website_products['Website'].values.tolist()
    prompt = f"Using the best of your knowledge and precise data analysis skills, for the value {company_in_row}, please choose the most appropriate value from the list below. Return only the exact value as given with no additional text. Important, if you do not see a fitting result, return N/A. Values are: {website_products_values}"
    
    # Call the OpenAI API to generate a response
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        temperature=.1
    )

    # Extract the generated response from the API response
    generated_text = response.choices[0].message.content

    companies.loc[index, 'Website'] = generated_text
display(companies)


,Company,Website
0,Apple,apple.com
1,LVMH,lvmh.com
2,Tesla,www.tesla.com
3,Meta,www.meta.com
4,Alphabet,N/A
5,Microsoft,N/A
6,Apple,apple.com
7,Tesla,www.tesla.com
8,Ford,N/A
9,ENI Energy,www.eni.it


## (3) Now just join the datasets and return table

In [5]:
companies_joined_with_website_products = pd.merge(companies, website_products, on='Website', how='left')
display(companies_joined_with_website_products)

,Company,Website,Primary product
0,Apple,apple.com,iPhone
1,LVMH,lvmh.com,Handbags
2,Tesla,www.tesla.com,Cars
3,Meta,www.meta.com,Social media
4,Alphabet,N/A,NaN
5,Microsoft,N/A,NaN
6,Apple,apple.com,iPhone
7,Tesla,www.tesla.com,Cars
8,Ford,N/A,NaN
9,ENI Energy,www.eni.it,Energy


## (4) Save the datasets as a file


In [ ]:
from save_file import save_new_csv
save_new_csv(file_path, companies_joined_with_website_products)

## (5) Fully built as a callable function
Call the function as follows:

```python
from llm_lookup import lookup

result = lookup(
    table1, # Pandas dataframe, required
    table2, # Pandas dataframe, required
    column_in_table1, # string, required
    column_in_table2, # string, required
    matching_context='', # string, optional
    openai_model='gpt-3.5-turbo', # string, optional
    temperature=0.1, # float, optional
    chunk_size=50, # int, optional
    return_stats=False # bool, optional
)
```

The optional parameters allow you to adjust the lookup for more tailored needs. You can choose the additional context, model, temperature, chunk size and whether or not to return stats on the lookup calls. The model, temperature and chunk size are an interplay that strongly depend on one another and the dataset given. Worth testing on a smaller data set and finetuning to see what works best for needs.

### (5.1) Example with just required variables

In [6]:
from llm_lookup import lookup
merged_df = lookup(companies,website_products,'Company','Website')

100%|██████████| 1/1 [00:03<00:00,  3.05s/it]


Now show me the results:

In [7]:
display(merged_df)

,Company,Website,Primary product
0,Apple,apple.com,iPhone
1,LVMH,lvmh.com,Handbags
2,Tesla,www.tesla.com,Cars
3,Meta,www.meta.com,Social media
4,Alphabet,N/A,NaN
5,Microsoft,N/A,NaN
6,Apple,apple.com,iPhone
7,Tesla,www.tesla.com,Cars
8,Ford,N/A,NaN
9,ENI Energy,www.eni.it,Energy


### (5.2) Example with optional variables
If you want to return stats, you also need to store them in a variable. 

In [8]:
from llm_lookup import lookup
merged_df, stats = lookup(companies,website_products,'Company','Website',
                          matching_context='Given a company name, you are to match the correct URL website as provided',
                          openai_model='gpt-4o',
                          temperature=0.2,
                          chunk_size=3,
                          return_stats=True)

100%|██████████| 3/3 [00:15<00:00,  5.04s/it]


Show me the results as well as stats:

In [9]:
display(merged_df)
display(stats)

,Company,Website,Primary product
0,Apple,apple.com,iPhone
1,LVMH,lvmh.com,Handbags
2,Tesla,www.tesla.com,Cars
3,Meta,www.meta.com,Social media
4,Alphabet,N/A,NaN
5,Microsoft,N/A,NaN
6,Apple,apple.com,iPhone
7,Tesla,www.tesla.com,Cars
8,Ford,N/A,NaN
9,ENI Energy,www.eni.it,Energy


{'API calls = ': 18, 'Cache calls = ': 12, 'Chunks = ': 3}

Save the file if you want:

In [ ]:
from save_file import save_new_csv
save_new_csv(file_path, merged_df)